# Using FAIRshake API

This notebook walks through using FAIRshake API with coreapi's python implementation, it works just as easily in javascript or any language with a coreapi implementation. Given that swagger is also exposed at <https://fairshake.cloud/v2/?format=openapi>, a swagger-based client can *also* be used.

For more information, refer to the documentation at https://fairshake.cloud/v2/coreapi/

Dependencies:
`pip install coreapi-cli`

In [ ]:
import coreapi

client = coreapi.Client()

url = 'https://fairshake.cloud/'

schema = client.get('https://fairshake.cloud/v2/coreapi/')

## Step 1. Authentication

If you don't already have a user registered with FAIRshake, you need to create one. This can be done manually at <https://fairshake.cloud/v2/accounts/signup/> or, through the API itself, this returns the API key for your account.

**Note**: Authentication is only required if you plan on *creating*/*changing* things. Read-access is available without authentication.

In [ ]:
# Set these to create/login to your own account
your_username = 'test'
your_password = 'my_test_pass'
# Set to True if you've already registered
registered = False

In [ ]:
if not registered:
    results = client.action(
        schema,
        ['auth', 'registration', 'create'],
        dict(
            username=your_username,
            password1=your_password,
            password2=your_password,
        ),
    )
    registered = True

If you already have an account, you can login through the API as well to obtain your API key.

In [ ]:
results = client.action(
    schema,
    ['auth', 'login', 'create'],
    dict(
        username=your_username,
        password=your_password,
    )
)
api_key = results['key']
results

Now that we have our API key, we can reinstantiate our Client with an authenticated transport layer.

In [ ]:
client = coreapi.Client(auth=coreapi.auth.TokenAuthentication(token=api_key, scheme='token'))
schema = client.get('https://fairshake.cloud/v2/coreapi/')

We can test that it worked by reading information about the logged in user.

In [ ]:
client.action(
    schema,
    ['auth', 'user', 'read'],
)

## Step 2. Project, Digital Object, Rubric, Metric Management

All elements expose themselves in the same way with a common set of attributes for search and identification with a few extra attributes distinguishing each element.

Here, quite simply, is the gist of these data models:

```python
class Identifiable:
  id: int
  url: str
  title: str
  description: str
  image: str
  tags: str
  type: ['', 'any', 'data', 'repo', 'test', 'tool']
  authors: Author[]

class Project(Identifiable):
  digital_objects: DigitalObject[]

class DigitalObject(Identifiable):
  rubrics: Rubric[]

class Rubric(Identifiable):
  license: str
  metrics: Metric[]

class Metric(Identifiable):
  license: str
  rationale: str
  principle: str
  fairmetrics: str
  fairsharing: str
```

Queries can be made by providing any of the parameters and we'll return the subset of the database which satisfies those parameter constraints. Though you use `title=something` we do a fuzzy search if it makes sense to do so. More find-tuned queries are actually supported by the API but not yet documented which would allow for django-style filters e.g. `title__contains=something`.

**Note**: Results are paginated; use `params={'page': n}` to go through pages

In [ ]:
# List all projects
client.action(schema, ['project', 'list'])

In [ ]:
# List all Digital objects of type Tool
client.action(schema, ['digital_object', 'list'], params=dict(type='tool'))

In [ ]:
# Create a metric
metric = client.action(schema, ['metric', 'create'], params=dict(
  title='My Metric',
  description='It has a url',
  type='url',
  tags='my project test',
  license='MIT',
  rationale='https://fairrationals.com/test',
  principle='F',
))
metric_id = metric['id']
metric

In [ ]:
# Create a rubric
rubric = client.action(schema, ['rubric', 'create'], params=dict(
  title='My Rubric',
  description='Rubric is great',
  tags='my project test',
  type='tool',
  license='MIT',
  metrics=[
      metric_id,
  ],
))
rubric_id = rubric['id']
rubric

In [ ]:
# Create a digital object
obj = client.action(schema, ['digital_object', 'create'], params=dict(
  url='http://my-objects.com/00001',
  title='My Object',
  description='Object is great',
  tags='my object test',
  type='tool',
  rubrics=[rubric_id],
))
obj_id = obj['id']
obj

In [ ]:
# Create a project
proj = client.action(schema, ['project', 'create'], params=dict(
  url='http://my-objects.com',
  title='My Project',
  description='Project is great',
  tags='my project test',
  digital_objects=[obj_id],
))
proj_id = proj['id']
proj

# Performing Assessments

```python
class Assessment:
  project: Project
  target: DigitalObject
  rubric: Rubric
  methodology: ['self', 'user', 'auto', 'test']
  answers: Answer[]

class Answer:
  metric: Metric
  answer: str
  comment: str
  url_comment: str
```

In [ ]:
# Create an assessment
client.action(schema, ['assessment', 'create'], params=dict(
  project=proj_id,
  target=obj_id,
  rubric=rubric_id,
  methodology='test',
  answers=[
      {
          'metric': metric_id,
          'answer': 'http://my_url.com',
      },
  ],
))

## Obtaining the score of a Digital Object

In [ ]:
score = client.action(schema, ['score', 'list'], params=dict(target=obj_id))
score

## Displaying FAIR insignia

The insignia client library exposes a function, `build_svg_from_score` which accepts a container and the query dict.

In [ ]:
import json
from IPython.display import HTML
HTML("""
<div
    id="insignia"
    data-target="%s"
    style="width: 40px; height: 40px; border: 0px solid black" />
""" % (obj_id))

In [ ]:
%%javascript
require(['https://fairshake.cloud/v2/static/scripts/insignia.js'], function(insignia) {
    var element = document.getElementById('insignia')
    insignia.build_svg_from_score(
        element,
        { target: element.getAttribute('data-target') }
    )
})

## Delete test account and associated creations

**WARNING**: Do not execute this code unless you don't need what you've created with this account.

In [ ]:
# Delete your account and associated objects
result = client.action(schema, ['auth', 'user', 'delete'])
result